<a href="https://colab.research.google.com/github/Deepakraj14-tamil/Leukemia-Data/blob/main/Plant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score


In [9]:
# Define the paths for the train and test data
train_data_dir = '/content/drive/MyDrive/Plant/Augmented Plant Diseases/Augmented Plant Diseases/train'
test_data_dir = '/content/drive/MyDrive/Plant/test'


In [10]:
# Define the input image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create the data generators for train and test data
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)


Found 69817 images belonging to 38 classes.
Found 33 images belonging to 1 classes.


In [11]:
# Define the number of classes
num_classes = train_generator.num_classes

In [12]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)

Epoch 1/10
1709/2181 [======================>.......] - ETA: 46:08 - loss: 0.9980 - accuracy: 0.7024

In [ ]:
# Evaluate the model
y_test_true = test_generator.classes
y_test_pred = model.predict(test_generator)
y_test_pred = tf.argmax(y_test_pred, axis=1)

In [ ]:
# Accuracy
accuracy = accuracy_score(y_test_true, y_test_pred)
print("Prediction accuracy: %.2f%%" % (accuracy * 100))

In [ ]:
# Bar graph of class-wise accuracy
class_wise_accuracy = accuracy_score(y_test_true, y_test_pred, normalize=False) / test_generator.samples
plt.figure(figsize=(8, 6))
plt.bar(class_labels, class_wise_accuracy)
plt.title('Class-wise Accuracy')
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Classification report
class_labels = list(train_generator.class_indices.keys())
classification_rep = classification_report(y_test_true, y_test_pred, target_names=class_labels)
print("Classification Report:\n", classification_rep)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test_true, y_test_pred)
plt.figure(figsize=(8, 8))
plt.imshow(cm, cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
# Training and validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#calculate the false positive rate (FPR), true positive rate (TPR), and thresholds for the ROC curve
y_test_probs = model.predict_proba(test_generator)[:, 1]  # Obtain predicted probabilities for the positive class
fpr, tpr, thresholds = roc_curve(y_test_true, y_test_probs)

In [ ]:
#Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
#Calculate the AUC score
auc_score = roc_auc_score(y_test_true, y_test_probs)
print('AUC Score:', auc_score)